In [3]:
import onnx
import tvm
from tvm import relay
from tvm.relay import transform
from tvm.contrib import graph_executor

onnx_model_path = "model.onnx"  
onnx_model = onnx.load(onnx_model_path)

shape_dict = {"input": [1, 3, 32, 32]}  
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

print("Original IR Module:")
print(mod)

mod = transform.InferType()(mod)
mod = transform.SimplifyInference()(mod)

mod = transform.FuseOps(fuse_opt_level=2)(mod)

mod = transform.AlterOpLayout()(mod)

print("\nUpdated IR Module after transformations:")
print(mod)

visualize_path = "updated_ir_module.json"
with open(visualize_path, "w") as f:
    f.write(mod.astext(show_meta_data=False))
print(f"Updated IR Module saved to {visualize_path}")

target = "llvm"  
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

dev = tvm.device(target, 0)
module = graph_executor.GraphModule(lib["default"](dev))

import numpy as np
input_data = np.random.rand(1, 3, 32, 32).astype("float32")
module.set_input("input",input_data)  
module.run()
output = module.get_output(0).asnumpy()
print("Inference Output:", output)


Original IR Module:
def @main(%input: Tensor[(1, 3, 32, 32), float32], %fc.weight: Tensor[(1000, 512), float32], %fc.bias: Tensor[(1000), float32], %onnx::Conv_193: Tensor[(64, 3, 7, 7), float32], %onnx::Conv_194: Tensor[(64), float32], %onnx::Conv_196: Tensor[(64, 64, 3, 3), float32], %onnx::Conv_197: Tensor[(64), float32], %onnx::Conv_199: Tensor[(64, 64, 3, 3), float32], %onnx::Conv_200: Tensor[(64), float32], %onnx::Conv_202: Tensor[(64, 64, 3, 3), float32], %onnx::Conv_203: Tensor[(64), float32], %onnx::Conv_205: Tensor[(64, 64, 3, 3), float32], %onnx::Conv_206: Tensor[(64), float32], %onnx::Conv_208: Tensor[(128, 64, 3, 3), float32], %onnx::Conv_209: Tensor[(128), float32], %onnx::Conv_211: Tensor[(128, 128, 3, 3), float32], %onnx::Conv_212: Tensor[(128), float32], %onnx::Conv_214: Tensor[(128, 64, 1, 1), float32], %onnx::Conv_215: Tensor[(128), float32], %onnx::Conv_217: Tensor[(128, 128, 3, 3), float32], %onnx::Conv_218: Tensor[(128), float32], %onnx::Conv_220: Tensor[(128, 128

In [4]:
import tvm
from tvm import relay
from tvm.contrib import graph_executor

# Load and build your model
# mod, params = relay.testing.mlp.get_workload(batch_size=1)
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target="llvm", params=params)

# Visualize IR
print(mod)
# Export to DOT (Graphviz)
dot_graph = tvm.ir.structural_hash(mod)
print(dot_graph)


def @main(%input: Tensor[(1, 3, 32, 32), float32], %fc.weight: Tensor[(1000, 512), float32], %fc.bias: Tensor[(1000), float32], %onnx::Conv_193: Tensor[(64, 3, 7, 7), float32], %onnx::Conv_194: Tensor[(64), float32], %onnx::Conv_196: Tensor[(64, 64, 3, 3), float32], %onnx::Conv_197: Tensor[(64), float32], %onnx::Conv_199: Tensor[(64, 64, 3, 3), float32], %onnx::Conv_200: Tensor[(64), float32], %onnx::Conv_202: Tensor[(64, 64, 3, 3), float32], %onnx::Conv_203: Tensor[(64), float32], %onnx::Conv_205: Tensor[(64, 64, 3, 3), float32], %onnx::Conv_206: Tensor[(64), float32], %onnx::Conv_208: Tensor[(128, 64, 3, 3), float32], %onnx::Conv_209: Tensor[(128), float32], %onnx::Conv_211: Tensor[(128, 128, 3, 3), float32], %onnx::Conv_212: Tensor[(128), float32], %onnx::Conv_214: Tensor[(128, 64, 1, 1), float32], %onnx::Conv_215: Tensor[(128), float32], %onnx::Conv_217: Tensor[(128, 128, 3, 3), float32], %onnx::Conv_218: Tensor[(128), float32], %onnx::Conv_220: Tensor[(128, 128, 3, 3), float32], %